# Scraping GialloZafferano

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import requests
import time
import pandas as pd
import ast
import re
import recordlinkage
from ydata_profiling import ProfileReport

In [ ]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--disable-notifications")

driver = webdriver.Chrome(options=chrome_options)

try:
    driver.get('https://www.giallozafferano.it/ricette-cat/page450')

    # Wait for the cookies popup to appear and click the "Continua senza accettare" button
    try:
        cookie_button = WebDriverWait(driver, 3).until(
            EC.element_to_be_clickable((By.XPATH, "//button[text()='Continua senza accettare']"))
        )
        cookie_button.click()
    except:
        print("No coockie' popup found.")

    tutte_ricette = []

    while True:
        # Scroll to the bottom of the page to load all recipes
        last_height = driver.execute_script("return document.body.scrollHeight")
        while True:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

        # Find all articles on the page
        articoli = driver.find_elements(By.CLASS_NAME, 'gz-card')

        # Iterate over all articles
        for i in range(len(articoli)):
            # Extract the title and category of the current article
            articoli = driver.find_elements(By.CLASS_NAME, 'gz-card')

            try:
                # Extract the title and category of the current article
                titolo = articoli[i].find_element(By.CLASS_NAME, 'gz-title').text
                categoria = articoli[i].find_element(By.CLASS_NAME, 'gz-category').text

                # Extract the link to the recipe and navigate to it
                link_ricetta = articoli[i].find_element(By.XPATH, ".//a[@title]").get_attribute('href')
                if link_ricetta is not None:
                    driver.get(link_ricetta)

                # Wait for the ingredients section to load
                wait = WebDriverWait(driver, 2)
                wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'gz-ingredients')))

                # Find the ingredients section and extract the list of ingredients
                ingredienti_section = driver.find_element(By.CLASS_NAME, 'gz-ingredients')
                ingredienti_list = ingredienti_section.find_elements(By.CLASS_NAME, 'gz-ingredient')

                # Extract the list of ingredients
                ingredienti_lista = [ingrediente.text for ingrediente in ingredienti_list]
                tutte_ricette.append({'titolo': titolo, 'categoria': categoria, 'ingredienti': ingredienti_lista})
                driver.back()
                time.sleep(3)
            except Exception as e:
                print(f"Error in article: {e}")

        # Click the "Next" button to navigate to the next page
        try:
            next_button = WebDriverWait(driver, 2).until(
                EC.element_to_be_clickable((By.CLASS_NAME, 'gz-arrow.next'))
            )
            next_button.click()
            time.sleep(2)
        except:
            print("There are no other pages left.")
            break
    for ricetta in tutte_ricette:
        ricetta['ingredienti'] = [ingrediente.replace("Olio extravergine d'oliva", "Olio extravergine di oliva") for ingrediente in ricetta['ingredienti']]

except Exception as e:
    print(f"Error: {e}")
finally:
    driver.quit()

ricette = pd.DataFrame(tutte_ricette)

Selecting only the recipes about Piatti Unici:

In [ ]:
ricette = ricette[ricette['categoria'] == 'PIATTI UNICI']

Parsing the ingredients:

In [ ]:
def parse_ingredient(ingredient):
    parts = ingredient.split()
    if len(parts) > 2 and (parts[-2].isdigit() and isinstance(parts[-1], str)):
        quantity = ' '.join(parts[-2:])
        name = ' '.join(parts[:-2])
    else:
        quantity = parts[-1]
        name = ' '.join(parts[:-1])
    return name, quantity

def convert_ingredient_list_to_dict(ingredient_list):
    ingredient_dict = {}
    for ingredient in ingredient_list:
        name, quantity = parse_ingredient(ingredient)
        ingredient_dict[name] = quantity
    return ingredient_dict

# Remove square brackets and single quotes from the 'ingredienti' column

ricette['ingredienti'] = ricette['ingredienti'].str.replace(r"\[|\]|'", "", regex=True)

# Split the 'ingredienti' column by ', ' to create a list of ingredients
ricette['ingredienti'] = ricette['ingredienti'].apply(lambda x: x.split(", "))

# Convert the list of ingredients to a dictionary
ricette['ingredienti'] = ricette['ingredienti'].apply(convert_ingredient_list_to_dict)
ricette.head()

Saving the dataset as a CSV and JSON:

In [ ]:
ricette.to_csv('ricette.csv', index=False)
ricette.to_json('ricette.json', orient='records')

In [ ]:
with open('Ricetteuniche.csv', 'r', encoding='utf-8') as file:
    content = file.read()

# Replace \u00bd with 0.5
content2 = content.replace('\u00bd', '0.5')

# Write the modified content back to the file
with open('Ricetteuniche.csv', 'w', encoding='utf-8') as file:
    file.write(content2)

Extracting the ingredients from the recipes and putting them in the right format:

In [ ]:
# Extract the ingredients from the 'ingredienti' column
def estrai_elementi(df):
    return [elemento for lista in df['ingredienti'] for elemento in lista]

ingredienti_estratti = estrai_elementi(ricette)
ingredients = pd.DataFrame(ingredienti_estratti)
ingredients[['ingrediente', 'quantita']] = ingredients[0].str.split(':', expand=True)
ingredients.drop(columns=[0], inplace=True)

In [ ]:
ingredients = ingredients['ingrediente'].unique()
ingredients.to_csv('ingredienti.csv', index=False)

 # SCRAPING OF CALORIES, PORTIONS AND COOKING TIME

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

base_url = "https://www.giallozafferano.it/ricette-cat/page{}/Piatti-Unici/"

# Lista per raccogliere tutte le ricette
all_recipes = []

# Iterare su tutte le 34 pagine
for page in range(1, 35):
    url = base_url.format(page)
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    recipe_cards = soup.find_all('div', class_='gz-card-content')

    for card in recipe_cards:
        title_element = card.find('h2', class_='gz-title')
        if title_element:
            recipe_name = title_element.get_text(strip=True)
        else:
            recipe_name = None

        bottom_data = card.find('ul', class_='gz-card-data bottom')
        portions = None
        cooking_time = None
        kcal = None

        if bottom_data:
            data_items = bottom_data.find_all('li', class_='gz-single-data-recipe')
            if data_items:
                for item in data_items:
                    text = item.get_text(strip=True)
                    if "min" in text:
                        cooking_time = text.replace("min", "").strip()
                    elif "Kcal" in text:
                        kcal = text.replace("Kcal", "").strip()
                    else:
                        portions = text.strip()

        # Aggiungi la ricetta alla lista
        all_recipes.append({
            "Recipe Name": recipe_name,
            "Portions": portions,
            "Cooking Time (min)": cooking_time,
            "Calories (Kcal)": kcal
        })

# Creare un dataframe pandas con i dati raccolti
df = pd.DataFrame(all_recipes)

# Mostrare il dataframe
print(df)


                               Recipe Name Portions Cooking Time (min)  \
0                         Insalata di riso        2                 35   
1                          Chili con carne        2             2 h 15   
2                         Gateau di patate        2             1 h 40   
3                        Paella de marisco        3             1 h 35   
4          Piadina romagnola fatta in casa        2                 34   
..                                     ...      ...                ...   
500                   Hosomaki vegetariano        4                 50   
501           Cornbread burger con porcino        3             1 h 40   
502                 Maki di kiwi e salmone        3                 42   
503               Demi baguette al salmone        2                 35   
504  Gateau di patate con zucca e taleggio        3             1 h 50   

    Calories (Kcal)  
0               660  
1               562  
2               317  
3               617  
4

In [ ]:
df.to_csv('/Users/matteosimeoni/Desktop/DATA_MAN/porzioni_calorie.csv', index=False)

#### VALORI CHE HANNO LE ORE IN PORZIONI

In [ ]:

def contains_number_h(portions):
    return bool(re.search(r'\d+ h', portions))

filtered_data = df[df['Portions'].apply(contains_number_h)]

filtered_data.shape
filtered_data


,Recipe Name,Portions,Cooking Time (min),Calories (Kcal)
19,Empanadas argentine,2 h,None,387
36,Fave e cicorie selvatiche,3 h,None,429
37,Ossobuco alla milanese con risotto giallo,4 h,None,1028
50,Tacos con carne,2 h,None,258
77,Polpettone,2 h,None,466
96,Hamburger vegetariano,2 h,None,501
105,Lasagne di pane carasau,2 h,None,794
120,Zighinì,3 h,None,548
175,Insalata di riso alla marinara,2 h,None,375
188,Moscardini alla busara,2 h,None,389


**da sistemare**:
- valori in ore
- valori con la prima colonna in h
- valori con stringhe nelle ore

Empanadas argentine=1
Fave e cicorie selvatiche = 4
Ossobuco alla milanese con risotto giallo=4
Tacos con carne=1
Polpettone= 6
Hamburger vegetariano=4
Lasagne di pane carasau= 9
Zighinì=6
Insalata di riso alla marinara=4
Moscardini alla busara=4
Gratin dauphinois=8
Tajine con verdure= 4
Empanadas con gorgonzola e porri4= 2
Burger di riso e barbabietola con salsa di Gra = 1

In [ ]:
import pandas as pd
import re

# Carica il dataset
file_path = '/Users/matteosimeoni/Desktop/DATA_MAN/porzioni_calorie.csv'
df = pd.read_csv(file_path)

# Dizionario per le ricette specifiche con i rispettivi valori di porzione
specific_portions = {
    "Empanadas argentine": 1,
    "Fave e cicorie selvatiche": 4,
    "Ossobuco alla milanese con risotto giallo": 4,
    "Tacos con carne": 1,
    "Polpettone": 6,
    "Hamburger vegetariano": 4,
    "Lasagne di pane carasau": 9,
    "Zighinì": 6,
    "Insalata di riso alla marinara": 4,
    "Moscardini alla busara": 4,
    "Gratin dauphinois": 8,
    "Tajine con verdure": 4,
    "Empanadas con gorgonzola e porri": 2,
    "Burger di riso e barbabietola con salsa di Gra": 1
}

# Funzione per sistemare i valori nelle colonne 'Portions' e 'Cooking Time (min)'
def fix_columns(row):
    if isinstance(row['Portions'], str) and 'h' in row['Portions']:
        row['Cooking Time (min)'] = row['Portions']
        row['Portions'] = specific_portions.get(row['Recipe Name'], 1)
    return row

# Applica la funzione per sistemare le colonne
df = df.apply(fix_columns, axis=1)

# Gestire i valori NaN sostituendoli con una stringa vuota
df['Cooking Time (min)'] = df['Cooking Time (min)'].fillna('')

# Funzione per convertire i tempi di cottura in minuti
def convert_to_minutes(time_str):
    if isinstance(time_str, str):
        # Convert hour and minute format to just minutes
        if 'h' in time_str:
            parts = time_str.split('h')
            hours = int(parts[0].strip())
            minutes = int(re.sub(r'\D', '', parts[1].strip())) if parts[1].strip() else 0
            return hours * 60 + minutes
        # Remove any non-digit characters and return the integer value
        return int(re.sub(r'\D', '', time_str)) if re.sub(r'\D', '', time_str) else 0
    return 0

# Applica la funzione alla colonna 'Cooking Time (min)'
df['Cooking Time (min)'] = df['Cooking Time (min)'].apply(convert_to_minutes)

# Salva il dataframe aggiornato in un nuovo file CSV
output_file_path = '/Users/matteosimeoni/Desktop/DATA_MAN/porzioni_calorie_sistemato.csv'
df.to_csv(output_file_path, index=False)

print("Il dataframe è stato sistemato e salvato come 'porzioni_calorie_sistemato.csv'.")


Il dataframe è stato sistemato e salvato come 'porzioni_calorie_sistemato.csv'.


# WEB SCRAPING CARREFOUR PRODUCTS

I've webscraped all the products name, their prices for kg and their brands name. (I've repeated the code for every categories of the products)

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from bs4 import BeautifulSoup
import requests

In [ ]:
import pandas as pd

# Set up Selenium web driver
driver = webdriver.Safari()

# Open the webpage
driver.get("https://www.carrefour.it/spesa-online/birra-e-liquori/?_gl=1*1eeymz2*_up*MQ..&gclid=Cj0KCQjwxqayBhDFARIsAANWRnQ-ziTsDUcj6cEwy158fN6S4l41Hs6cRXFDIEQdCXhCh0gTSUZBkO4aAvGqEALw_wcB&gclsrc=aw.ds")

# Define a function to scroll the page
def scroll_page(driver):
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(10)  # Wait for content to load
        new_height = driver.execute_script("return document.body.scrollHeight")

        if new_height == last_height:
            break

        last_height = new_height

# Scroll the page until the end
scroll_page(driver)

# Get the page source after scrolling
html = driver.page_source

# Parse the HTML with BeautifulSoup
soup = BeautifulSoup(html, "html.parser")

# Now you can use BeautifulSoup to extract the data you need
# For example:
# data = soup.find_all("div", class_="some-class")

# Close the browser
driver.quit()

product_items = soup.find_all('div', class_='product-item-with-tooltip')

# Initialize an empty dictionary to store product data
product_data = {}

# Iterate over each product item
for product in product_items:
    # Extract product description
    description_elem = product.find('span', class_='tile-description')
    description = description_elem.get_text(strip=True) if description_elem else None

    # Extract product brand if it exists
    brand_elem = product.find('span', class_='brand')
    brand = brand_elem.get_text(strip=True) if brand_elem else None

    # Extract product price if it exists
    price_elem = product.find('span', class_='unit-price')
    if price_elem:
        # Extract the price value
        price_text = price_elem.get_text(strip=True)
        # Split the text to get only the price part
        price_parts = price_text.split("€ ")[-1].split()
        price = price_parts[0]  # Extracting the price
        unit = ' '.join(price_parts[1:])  # Joining remaining parts
    else:
        price = None

    # Create a dictionary entry for the product
    if description:
        product_data[description] = {'brand': brand, 'price': price}

# Print the product data dictionary
print(product_data)

# Create a DataFrame from the product data dictionary
df = pd.DataFrame.from_dict(product_data, orient='index').reset_index()

# Rename the 'index' column to 'product_name'
df = df.rename(columns={'index': 'product_name'})

# Save the DataFrame to a CSV file
csv_file = 'product_data_BIRRA E LIQUORI.csv'
df.to_csv(csv_file, index=False)

print(f"Dati salvati con successo nel file {csv_file}")

# DATA EXPLORATION

In [ ]:
df = pd.read_csv("Ricetteuniche.csv")
report = ProfileReport(df, title="Profiling Report")
report

In [ ]:
df = pd.read_csv("prodotti_concatenati.csv")
report = ProfileReport(df, title="Profiling Report")
report

# DATA QUALITY

#### Extraction of Units of Measurement to Assign or Replace

In [ ]:
import json
import re

# Load the JSON data
with open('Ricetteuniche6.json', 'r') as file:
    data = json.load(file)

# Define a function to check if the quantity is in the correct format
def is_correct_format(quantity):
    return re.match(r'^\d+(\.\d+)?\s*g$', quantity) is not None

# Extract unique quantities of ingredients excluding those in the correct format
unique_quantities = set()
for recipe in data:
    for ingredient, quantity in recipe.get('ingredienti', {}).items():
        if not is_correct_format(quantity):
            quantity_words = quantity.split()
            # If it's a single word and not a number, add to unique quantities
            if len(quantity_words) == 1 and not quantity.replace('.', '', 1).isdigit():
                unique_quantities.add(quantity)
            # If it's a multi-word quantity, add the last word (unit)
            elif len(quantity_words) > 1:
                unique_quantities.add(quantity_words[-1])

unique_quantities_list = sorted(unique_quantities)
unique_quantities_list


q.b. -> 3 g (essendo solo per spezie)

q.b." -> 3 g

ml -> g

gr -> g

g" -> g

gocce -> 1 g

costa -> 5 g

coste -> 5 g x numero

l -> 1000 g x numero

cucchiai -> 10 g x numero

cucchiaini -> 5 g x numero

cucchiai" -> 10 g x numero

cucchiaino -> 10 g

cucchiaino" -> 10 g

cucchiaio -> 5 g

pizzico -> 3 g

rametto -> 5 g

foglie -> 5 g x numero

spicchio -> 5 g

spicchi -> 5 g x numero

foglio -> 20 g

bacca -> 3 g

ciuffo -> 10 g (riguarda il prezzemolo, circa 10 g)

mazzo -> 10 g  (riguarda basilico, salvia e menta, circa 10 g)

foglia -> 5 g

fogliolina -> 3 g

fette -> 20 g x numero (quasi sempre pane)

fette" -> 20 g x numero

rametto -> 3 g

rametti -> 3g x numero

bicchiere -> 250 g

filetti" -> 100 g x numero

filetto" > 100 g

radice -> 10 g

mazzetto -> 10 g


NUMERI SENZA g

0.5" -> 0.5 g
 1" -> 1 g
100 -> 100 g
 2" -> 2 g
'4" -> 4 g
'5" -> 5 g

In [ ]:
import json
import re

# Load the JSON data
with open('/Users/matteosimeoni/Desktop/DATA_MAN/Ricetteuniche6.json', 'r') as file:
    data = json.load(file)

# Conversion table for specific units
conversion_table = {
    "q.b.": "3 g",
    "q.b.\"": "3 g",
    "ml": "g",
    "gr": "g",
    "g\"": "g",
    "gocce": "1 g",
    "costa": "5 g",
    "coste": "5 g",
    "l": "1000 g",
    "cucchiai": "10 g",
    "cucchiaini": "5 g",
    "cucchiai\"": "10 g",
    "cucchiaino": "10 g",
    "cucchiaino\"": "10 g",
    "cucchiaio": "5 g",
    "pizzico": "3 g",
    "rametto": "5 g",
    "foglie": "5 g",
    "spicchio": "5 g",
    "spicchi": "5 g",
    "foglio": "20 g",
    "bacca": "3 g",
    "ciuffo": "10 g",
    "mazzo": "10 g",
    "foglia": "5 g",
    "fogliolina": "3 g",
    "fette": "20 g",
    "fette\"": "20 g",
    "rametti": "3 g",
    "rametto": "3 g",
    "bicchiere": "250 g",
    "filetti\"": "100 g",
    "filetto\"": "100 g",
    "radice": "10 g",
    "mazzetto": "10 g",
    "kg": "1000 g",
    "0.5\"": "0.5 g",
    "1\"": "1 g",
    "100": "100 g",
    "2\"": "2 g",
    "4\"": "4 g",
    "5\"": "5 g"
}

# List of values to remove the entire recipe
values_to_remove_recipe = [
    '(abbattuto', '(secco', 'Modena', 'abbattuto', 'grossolanamente', 'nero',
    'peperoncino', 'qualche', 'salmone', 'secco'
]

# Define a function to clean quantities
def clean_quantity(quantity):
    # Replace ',' with '.'
    quantity = quantity.replace(',', '.')

    # Check for specific conversions
    for unit, conversion in conversion_table.items():
        if unit in quantity:
            if " x " in conversion:  # handle multiplicative conversions
                number_match = re.match(r'(\d+(\.\d+)?)\s*' + unit, quantity)
                if number_match:
                    number = float(number_match.group(1))
                    return f"{number * float(conversion.split()[0])} g"
            else:
                return conversion if " " not in conversion else f"{conversion}"

    return quantity

# Clean all quantities in the data
cleaned_data = []
for recipe in data:
    remove_recipe = False
    cleaned_ingredients = {}
    for ingredient, quantity in recipe.get('ingredienti', {}).items():
        if quantity in values_to_remove_recipe:
            remove_recipe = True
            break
        cleaned_ingredients[ingredient] = clean_quantity(quantity)

    if not remove_recipe:
        recipe['ingredienti'] = cleaned_ingredients
        cleaned_data.append(recipe)

# Save the cleaned data
cleaned_file_path = '/Users/matteosimeoni/Desktop/DATA_MAN/Ricetteuniche6_cleaned.json'
with open(cleaned_file_path, 'w') as file:
    json.dump(cleaned_data, file, ensure_ascii=False, indent=4)

cleaned_file_path


In [ ]:
import json
import re

# Load the cleaned JSON data
file_path = '/Users/matteosimeoni/Desktop/DATA_MAN/Ricetteuniche6_cleaned.json'
with open(file_path, 'r') as file:
    data = json.load(file)

# Function to check if the quantity is in the correct format (number followed by 'g')
def is_correct_format(quantity):
    return re.match(r'^\d+(\.\d+)?\s*g$', quantity) is not None

# Collect records with incorrect quantities
incorrect_records = []
for recipe in data:
    incorrect_ingredients = {ingredient: quantity for ingredient, quantity in recipe['ingredienti'].items() if not is_correct_format(quantity)}
    if incorrect_ingredients:
        incorrect_records.append({
            "titolo": recipe["titolo"],
            "categoria": recipe["categoria"],
            "ingredienti": incorrect_ingredients
        })

# Save the incorrect records to a new JSON file
incorrect_file_path_final = '/Users/matteosimeoni/Desktop/DATA_MAN/ingredients_incorrect_final.json'
with open(incorrect_file_path_final, 'w') as file:
    json.dump(incorrect_records, file, ensure_ascii=False, indent=4)

incorrect_file_path_final


#### Elimination of Ingredients without Quantity

In [ ]:
import json

# Load the JSON data
file_path = '/Users/matteosimeoni/Desktop/DATA_MAN/Ricetteuniche6_cleaned.json'
with open(file_path, 'r') as file:
    data = json.load(file)

# Function to check if the quantity is invalid (only "g" or empty)
def is_invalid_quantity(quantity):
    return quantity == "g" or quantity == ""

# Create a new list to hold the cleaned data
cleaned_data = []
for recipe in data:
    cleaned_ingredients = {ingredient: quantity for ingredient, quantity in recipe['ingredienti'].items() if not is_invalid_quantity(quantity)}
    if cleaned_ingredients:
        cleaned_data.append({
            "titolo": recipe["titolo"],
            "categoria": recipe["categoria"],
            "ingredienti": cleaned_ingredients
        })

# Save the cleaned data to a new JSON file
cleaned_file_path_final = '/Users/matteosimeoni/Desktop/DATA_MAN/Ricetteuniche6_cleaned_final.json'
with open(cleaned_file_path_final, 'w') as file:
    json.dump(cleaned_data, file, ensure_ascii=False, indent=4)

cleaned_file_path_final


#### CREATION OF A JSON FILE WITH INCORRECT RECORDS

In [ ]:
import json
import re

# Load the cleaned JSON data
file_path = '/Users/matteosimeoni/Desktop/DATA_MAN/Ricetteuniche6_cleaned_final.json'
with open(file_path, 'r') as file:
    data = json.load(file)

# Function to check if the quantity is in the correct format (number followed by 'g')
def is_correct_format(quantity):
    return re.match(r'^\d+(\.\d+)?\s*g$', quantity) is not None

# Collect records with incorrect quantities
incorrect_records = []
for recipe in data:
    incorrect_ingredients = {ingredient: quantity for ingredient, quantity in recipe['ingredienti'].items() if not is_correct_format(quantity)}
    if incorrect_ingredients:
        incorrect_records.append({
            "titolo": recipe["titolo"],
            "categoria": recipe["categoria"],
            "ingredienti": incorrect_ingredients
        })

# Save the incorrect records to a new JSON file
incorrect_file_path_final = '/Users/matteosimeoni/Desktop/DATA_MAN/ingredients_incorrect_final.json'
with open(incorrect_file_path_final, 'w') as file:
    json.dump(incorrect_records, file, ensure_ascii=False, indent=4)

incorrect_file_path_final


# INGREDIENTS DATASET

I've extracted all the names od the ingridients of the recipes

In [ ]:
import pandas as pd
import ast
import re

In [ ]:
df=pd.read_csv('Ricetteuniche2.csv')
df

,titolo,categoria,ingredienti,ingredienti_dict
0,Masala di ceci,PIATTI UNICI,"['Ceci precotti (peso sgocciolato) 500 g', 'La...","{'Ceci precotti (peso sgocciolato)': '500 g', ..."
1,Bacalhau à Braz,PIATTI UNICI,"['Baccalà sotto sale 400 g', 'Patate 500 g', '...","{'Baccalà sotto sale': '400 g', 'Patate': '500..."
2,Pasticcio di patate e pancetta,PIATTI UNICI,"['Patate tutte della stessa grandezza 1 kg', '...",{'Patate tutte della stessa grandezza': '1 kg'...
3,Burrito con verdure,PIATTI UNICI,"['Tortillas di farina 8', 'Fagioli borlotti pr...","{'Tortillas di farina': '8', 'Fagioli borlotti..."
4,Pasticcio di pollo e patate,PIATTI UNICI,"['Patate 600 g', 'Pollo (non disossato e senza...","{'Patate': '600 g', 'Pollo (non disossato e se..."
...,...,...,...,...
233,Gunkanmaki,PIATTI UNICI,"['Riso per sushi cotto 25 g', 'Alga nori q.b.'...","{'Riso per sushi cotto': '25 g', 'Alga nori': ..."
234,"Piadina con 'nduja, salsiccia e pomodorini",PIATTI UNICI,"['Piadine Sfogliatissime 2', 'Salsiccia 400 g'...","{'Piadine Sfogliatissime': '2', 'Salsiccia': '..."
235,L'Affumicato,PIATTI UNICI,"['Petto di pollo 100% italiano 300 g', 'Lattug...","{'Petto di pollo 100% italiano': '300 g', 'Lat..."
236,"Piadina con crudo, pesto di fave e indivia",PIATTI UNICI,"['Piadine integrali 2', 'Indivia belga 2', 'Pe...","{'Piadine integrali': '2', 'Indivia belga': '2..."


In [ ]:
df['ingredienti_dict'] = df['ingredienti_dict'].apply(ast.literal_eval)

In [ ]:
df['ingredienti_dict']

0      {'Ceci precotti (peso sgocciolato)': '500 g', ...
1      {'Baccalà sotto sale': '400 g', 'Patate': '500...
2      {'Patate tutte della stessa grandezza': '1 kg'...
3      {'Tortillas di farina': '8', 'Fagioli borlotti...
4      {'Patate': '600 g', 'Pollo (non disossato e se...
                             ...                        
233    {'Riso per sushi cotto': '25 g', 'Alga nori': ...
234    {'Piadine Sfogliatissime': '2', 'Salsiccia': '...
235    {'Petto di pollo 100% italiano': '300 g', 'Lat...
236    {'Piadine integrali': '2', 'Indivia belga': '2...
237    {'"Burrata (4 da 125 g luna)': '500 g"', 'Radi...
Name: ingredienti_dict, Length: 238, dtype: object

In [ ]:
# Funzione per trasformare il dizionario in una lista
def dict_to_list(diz):
    return [f"{key}: {value}" for key, value in diz.items()]


In [ ]:
df['ingredienti_lista'] = df['ingredienti_dict'].apply(dict_to_list)

df

,titolo,categoria,ingredienti,ingredienti_dict,ingredienti_lista
0,Masala di ceci,PIATTI UNICI,"['Ceci precotti (peso sgocciolato) 500 g', 'La...","{'Ceci precotti (peso sgocciolato)': '500 g', ...","[Ceci precotti (peso sgocciolato): 500 g, Latt..."
1,Bacalhau à Braz,PIATTI UNICI,"['Baccalà sotto sale 400 g', 'Patate 500 g', '...","{'Baccalà sotto sale': '400 g', 'Patate': '500...","[Baccalà sotto sale: 400 g, Patate: 500 g, Lat..."
2,Pasticcio di patate e pancetta,PIATTI UNICI,"['Patate tutte della stessa grandezza 1 kg', '...",{'Patate tutte della stessa grandezza': '1 kg'...,"[Patate tutte della stessa grandezza: 1 kg, Ci..."
3,Burrito con verdure,PIATTI UNICI,"['Tortillas di farina 8', 'Fagioli borlotti pr...","{'Tortillas di farina': '8', 'Fagioli borlotti...","[Tortillas di farina: 8, Fagioli borlotti prec..."
4,Pasticcio di pollo e patate,PIATTI UNICI,"['Patate 600 g', 'Pollo (non disossato e senza...","{'Patate': '600 g', 'Pollo (non disossato e se...","[Patate: 600 g, Pollo (non disossato e senza p..."
...,...,...,...,...,...
233,Gunkanmaki,PIATTI UNICI,"['Riso per sushi cotto 25 g', 'Alga nori q.b.'...","{'Riso per sushi cotto': '25 g', 'Alga nori': ...","[Riso per sushi cotto: 25 g, Alga nori: q.b., ..."
234,"Piadina con 'nduja, salsiccia e pomodorini",PIATTI UNICI,"['Piadine Sfogliatissime 2', 'Salsiccia 400 g'...","{'Piadine Sfogliatissime': '2', 'Salsiccia': '...","[Piadine Sfogliatissime: 2, Salsiccia: 400 g, ..."
235,L'Affumicato,PIATTI UNICI,"['Petto di pollo 100% italiano 300 g', 'Lattug...","{'Petto di pollo 100% italiano': '300 g', 'Lat...","[Petto di pollo 100% italiano: 300 g, Lattuga ..."
236,"Piadina con crudo, pesto di fave e indivia",PIATTI UNICI,"['Piadine integrali 2', 'Indivia belga 2', 'Pe...","{'Piadine integrali': '2', 'Indivia belga': '2...","[Piadine integrali: 2, Indivia belga: 2, Pecor..."


In [ ]:
# estrazione degli elementi delle liste in un unica lista
def estrai_elementi(df, colonna):
    return [elemento for lista in df[colonna] for elemento in lista]

# Utilizzo della funzione per estrarre tutti gli elementi
ingredienti_estratti = estrai_elementi(df, 'ingredienti_lista')

In [ ]:
ingredienti_estratti

['Ceci precotti (peso sgocciolato): 500 g',
 'Latte di cocco intero: 300 g',
 'Pomodori ramati: 1',
 'Cipolle bianche: ½',
 'Aglio: 1 spicchio',
 'Zenzero fresco: q.b.',
 'Peperoncino fresco: 1',
 'Garam Masala: 1 cucchiaio',
 'Cumino in polvere: 1 cucchiaio',
 'Curcuma in polvere: 1 cucchiaio',
 'Olio extravergine di oliva: q.b.',
 'Sale fino: q.b.',
 'Coriandolo: q.b.',
 'Succo di lime: q.b.',
 'Baccalà sotto sale: 400 g',
 'Patate: 500 g',
 'Latte intero: 600 ml',
 'Cipolle bianche: 3',
 'Olive nere denocciolate: q.b.',
 'Uova: 4',
 'Aglio: 1 spicchio',
 'Prezzemolo: 1 rametto',
 'Olio extravergine di oliva: q.b.',
 'Sale fino: q.b.',
 'Pepe nero: q.b.',
 'Olio di semi di arachide: q.b.',
 'Patate tutte della stessa grandezza: 1 kg',
 'Cipolle bianche: 400 g',
 'Pancetta affumicata: 250 g',
 'Asiago grattugiato: 200 g',
 'Sale fino: 1 pizzico',
 'Pepe nero: 1 pizzico',
 'Timo: 4 rametti',
 'Farina 00: 30 g',
 'Burro: 30 g',
 'Latte intero: 500 ml',
 'Noce moscata da grattugiare: q.b

In [ ]:
ing=pd.DataFrame(ingredienti_estratti)
ing

,0
0,Ceci precotti (peso sgocciolato): 500 g
1,Latte di cocco intero: 300 g
2,Pomodori ramati: 1
3,Cipolle bianche: ½
4,Aglio: 1 spicchio
...,...
2963,Olio extravergine di oliva: 100 g
2964,Basilico: 5 foglie
2965,Timo: 4 rametti
2966,Sale fino: q.b.


### SEPARATE QUANTITY FROM NAME AND EXTRACT THE NAMES

In [ ]:
ing[['ingrediente', 'quantita']] = ing[0].str.split(':', expand=True)
ing.drop(columns=[0], inplace=True)
ing

,ingrediente,quantita
0,Ceci precotti (peso sgocciolato),500 g
1,Latte di cocco intero,300 g
2,Pomodori ramati,1
3,Cipolle bianche,½
4,Aglio,1 spicchio
...,...,...
2963,Olio extravergine di oliva,100 g
2964,Basilico,5 foglie
2965,Timo,4 rametti
2966,Sale fino,q.b.


In [ ]:
nomi_ing=pd.DataFrame(ing['ingrediente'].unique())
nomi_ing

,0
0,Ceci precotti (peso sgocciolato)
1,Latte di cocco intero
2,Pomodori ramati
3,Cipolle bianche
4,Aglio
...,...
859,Fave
860,"""Burrata (4 da 125 g luna)"
861,Radicchio lungo
862,Cachi mela


In [ ]:
nomi_ing.rename(columns={0: 'ingrediente'}, inplace=True)
nomi_ing

,ingrediente
0,Ceci precotti (peso sgocciolato)
1,Latte di cocco intero
2,Pomodori ramati
3,Cipolle bianche
4,Aglio
...,...
859,Fave
860,"""Burrata (4 da 125 g luna)"
861,Radicchio lungo
862,Cachi mela


In [ ]:
# nomi_ing.to_csv('nomi_ing.csv',index=False)

# MATCHING

In [ ]:
df_ingredienti=pd.read_csv('nomi_ing.csv')
df_prodotti = pd.read_csv('prodotti_concatenati.csv')

In [ ]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


# Definisci le parole inutili come un set per ricerche più veloci
PAROLE_INUTILI = set([
    'il', 'lo', 'la', 'i', 'gli', 'le', 'un', 'uno', 'una', 'del', 'dello', 'della', 'dei', 'degli', 'delle', 'e', 'di',
    'con', 'per', 'al', 'a', 'da', 'in', 'su', 'tra', 'fra', 'ma', 'anche', 'o', 'né', 'neanche',
    'nondimeno', 'tuttavia', 'anzi', 'benché', 'sebbene', 'temperatura', 'ambiente', 'fresco', 'Macinata',
    'baccello', 'Classic', 'macinato', 'grani', 'sacchetto', 'Filiera', 'Qualità', 'dorate', 'bio', 'fiocchi',
    'Mercato', 'sgocciolato'
])

# Estrarre parole dei brand
brand_parole = set()
for brand in df_prodotti['brand'].unique():
    brand_parole.update(re.findall(r'\b\w+\b', brand.lower()))

parole_inutili_brand = PAROLE_INUTILI | brand_parole
parole_inutili_brand -= {'panna', 'sedano', 'nutella'}

def pulisci_testo(test: str, parole_inutili: set) -> str:
    # Rimuove tutto tra parentesi
    parole = re.sub(r'\([^)]*\)', '', test)
    # Divide la stringa in parole e converte a minuscolo
    parole = parole.lower().split()
    # Filtra le parole inutili ignorando la capitalizzazione
    parole_filtrate = [parola for parola in parole if parola not in parole_inutili]
    return ' '.join(parole_filtrate)

# Pulire ingredienti e prodotti
df_ingredienti['nome_pulito'] = df_ingredienti['ingrediente'].apply(lambda x: pulisci_testo(x, PAROLE_INUTILI))
df_prodotti['nome_pulito'] = df_prodotti['product_name'].apply(lambda x: pulisci_testo(x, parole_inutili_brand))

# Precalcolare i vettori per gli ingredienti e i prodotti usando TfidfVectorizer
vectorizer = TfidfVectorizer()
ingredienti_vettori = vectorizer.fit_transform(df_ingredienti['nome_pulito'])
prodotti_vettori = vectorizer.transform(df_prodotti['nome_pulito'])

# Lista per tenere traccia delle righe del DataFrame risultante
righe_df = []

# Itera attraverso ogni ingrediente e prodotto usando cosine similarity
similarita = cosine_similarity(ingredienti_vettori, prodotti_vettori)

for i, ingrediente in enumerate(df_ingredienti['ingrediente']):
    corrispondenze = []
    for j, prodotto in enumerate(df_prodotti['product_name']):
        indice_similarita = similarita[i, j]
        if indice_similarita > 0.7:
            corrispondenze.append({'prodotto': prodotto, 'indice_similarita': indice_similarita})

    if not corrispondenze:
        righe_df.append({'ingrediente': ingrediente, 'corrispondenza': None, 'indice_similarita': None})
    else:
        for corrispondenza in corrispondenze:
            righe_df.append({'ingrediente': ingrediente, 'corrispondenza': corrispondenza['prodotto'], 'indice_similarita': corrispondenza['indice_similarita']})

# Costruisci un DataFrame dalle righe
df_merged = pd.DataFrame(righe_df)

# Visualizza il DataFrame risultante
df_merged.head(20)


,ingrediente,corrispondenza,indice_similarita
0,Pomodori ramati,None,NaN
1,Garam Masala,Save Quinoa Noodles Curry Masala 70 g,0.408248
2,Garam Masala,sabita Tikka Masala Simmer Sauce 200 ml,0.500000
3,Cumino in polvere,Knorr Aromat Insaporitore in Polvere 90 g,0.724946
4,Cumino in polvere,PERUGINA Cacao Amaro in Polvere 75g,0.724946
5,Cumino in polvere,Carrefour Bio Cacao Amaro in polvere 75 g,0.724946
6,Curcuma in polvere,Hari&Co Polpettine Vegetali di Ceci Patate Dol...,0.730685
7,Curcuma in polvere,bella vita Protein Mango Curcuma 235 g,0.730685
8,Curcuma in polvere,Carrefour Classic Curcuma Macinata 38 g,0.730685
9,Curcuma in polvere,Knorr Aromat Insaporitore in Polvere 90 g,0.682715


In [ ]:
df_merged

,ingrediente,corrispondenza,indice_similarita
0,Pomodori ramati,None,NaN
1,Garam Masala,Save Quinoa Noodles Curry Masala 70 g,0.408248
2,Garam Masala,sabita Tikka Masala Simmer Sauce 200 ml,0.500000
3,Cumino in polvere,Knorr Aromat Insaporitore in Polvere 90 g,0.724946
4,Cumino in polvere,PERUGINA Cacao Amaro in Polvere 75g,0.724946
...,...,...,...
7607,Uova di lompo (succedaneo caviale) o di salmon...,Carrefour Classic Uova di Lompo Nere 50 g,0.535420
7608,Uova di lompo (succedaneo caviale) o di salmon...,Mowi Gourmet Infusions Tranci di Salmone Marin...,0.430154
7609,Uova di lompo (succedaneo caviale) o di salmon...,Carrefour Classic Uova di Lompo Rosse 50 g,0.663122
7610,Uova di lompo (succedaneo caviale) o di salmon...,Cubetti di Salmone Norvegese Affumicati a Cald...,0.430154


In [ ]:
#df_merged.to_csv('matched.csv',index=False)

# CHECK NULL VALUES AND SAVE THEM TO A CSV

In [ ]:
nulli=df_merged[df_merged['corrispondenza'].isnull()]

In [ ]:
nulli

,ingrediente,corrispondenza,indice_similarita
13,Pomodori ramati,None,NaN
38,Garam Masala,None,NaN
39,Cumino in polvere,None,NaN
40,Curcuma in polvere,None,NaN
64,Baccalà sotto sale,None,NaN
...,...,...,...
4487,Insalata trocadero,None,NaN
4488,Aragosta 4,None,NaN
4498,Champagne,None,NaN
4499,Riso pilaf,None,NaN


In [ ]:
#nulli.to_csv('nulli.csv',index=False)

# MERGED CON I PREZZI

In [ ]:
df_matched

,ingrediente,corrispondenza,indice_similarita
0,Ceci precotti (peso sgocciolato),Knorr Zuppa di Ceci 545 g,0.734605
1,Ceci precotti (peso sgocciolato),Valfrutta Ceci 360 g,0.734605
2,Ceci precotti (peso sgocciolato),Carrefour Bio Hummus di Ceci 130 g,0.734605
3,Ceci precotti (peso sgocciolato),Carrefour Ceci cotti al Vapore 3 x 150 g,0.734605
4,Ceci precotti (peso sgocciolato),Carrefour Ceci 330 g,0.734605
...,...,...,...
4557,Mele Golden,Mele Golden 900 g,0.763971
4558,Mele Golden,Mele Golden 3 kg,0.763971
4559,Mele Golden,Mele golden Melinda 3 kg,0.763971
4560,Mele Golden,Mele Golden,0.763971


In [ ]:
df_matched = pd.read_csv('matched.csv')
df_prodotti_concatenati = pd.read_csv('prodotti_concatenati.csv')

# Effettua il merge tenendo solo le osservazioni left
df_ing_prezzi = pd.merge(df_matched, df_prodotti_concatenati, left_on='corrispondenza', right_on='product_name', how='left')
df_ing_prezzi

,ingrediente,corrispondenza,indice_similarita,product_name,brand,price,source_file
0,Ceci precotti (peso sgocciolato),Knorr Zuppa di Ceci 545 g,0.734605,Knorr Zuppa di Ceci 545 g,Knorr,"4,13",PASTA_FARINA_RISO
1,Ceci precotti (peso sgocciolato),Valfrutta Ceci 360 g,0.734605,Valfrutta Ceci 360 g,Valfrutta,"5,17",CONDIMENTI_CONSERVE
2,Ceci precotti (peso sgocciolato),Valfrutta Ceci 360 g,0.734605,Valfrutta Ceci 360 g,Valfrutta,"5,17",FRUTTA_VERDURA
3,Ceci precotti (peso sgocciolato),Carrefour Bio Hummus di Ceci 130 g,0.734605,Carrefour Bio Hummus di Ceci 130 g,Carrefour Bio,"20,69",CONDIMENTI_CONSERVE
4,Ceci precotti (peso sgocciolato),Carrefour Ceci cotti al Vapore 3 x 150 g,0.734605,Carrefour Ceci cotti al Vapore 3 x 150 g,Carrefour,"4,74",CONDIMENTI_CONSERVE
...,...,...,...,...,...,...,...
4975,Mele Golden,Mele Golden 900 g,0.763971,Mele Golden 900 g,Frutta,"1,99",FRUTTA_VERDURA
4976,Mele Golden,Mele Golden 3 kg,0.763971,Mele Golden 3 kg,Frutta,"1,33",FRUTTA_VERDURA
4977,Mele Golden,Mele golden Melinda 3 kg,0.763971,Mele golden Melinda 3 kg,Melinda,"1,79",FRUTTA_VERDURA
4978,Mele Golden,Mele Golden,0.763971,Mele Golden,Mele,"0,98",FRUTTA_VERDURA


In [ ]:
df_ing_prezzi= df_ing_prezzi.drop(columns=['corrispondenza', 'indice_similarita', 'brand'])

In [ ]:
#df_ing_prezzi.to_csv('matched_finale.csv')

# CREATION OF THE PRICE RANGE DATASET BUT WITH NULL VALUES

In [ ]:
import pandas as pd

In [ ]:
data=pd.read_csv('matched_finale.csv')

In [ ]:
data

,Unnamed: 0,ingrediente,product_name,price,source_file
0,1,Ceci precotti (peso sgocciolato),Valfrutta Ceci 360 g,"5,17",CONDIMENTI_CONSERVE
1,2,Ceci precotti (peso sgocciolato),Valfrutta Ceci 360 g,"5,17",FRUTTA_VERDURA
2,4,Ceci precotti (peso sgocciolato),Carrefour Ceci cotti al Vapore 3 x 150 g,"4,74",CONDIMENTI_CONSERVE
3,5,Ceci precotti (peso sgocciolato),Carrefour Ceci cotti al Vapore 3 x 150 g,"4,74",FRUTTA_VERDURA
4,6,Ceci precotti (peso sgocciolato),Carrefour Ceci 330 g,"5,07",CONDIMENTI_CONSERVE
...,...,...,...,...,...
4476,4975,Mele Golden,Mele Golden 900 g,"1,99",FRUTTA_VERDURA
4477,4976,Mele Golden,Mele Golden 3 kg,"1,33",FRUTTA_VERDURA
4478,4977,Mele Golden,Mele golden Melinda 3 kg,"1,79",FRUTTA_VERDURA
4479,4978,Mele Golden,Mele Golden,"0,98",FRUTTA_VERDURA


In [ ]:
data['price']=data['price'].str.replace('.', '').str.replace(',', '.').astype(float)

/var/folders/wl/qhbk6mrs6dx_2nxjr_p99fj00000gn/T/ipykernel_6800/2597366130.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data['price']=data['price'].str.replace('.', '').str.replace(',', '.').astype(float)


In [ ]:
data['price']

0       5.17
1       5.17
2       4.74
3       4.74
4       5.07
        ... 
4476    1.99
4477    1.33
4478    1.79
4479    0.98
4480    3.11
Name: price, Length: 4481, dtype: float64

In [ ]:
data

,Unnamed: 0,ingrediente,product_name,price,source_file
0,1,Ceci precotti (peso sgocciolato),Valfrutta Ceci 360 g,5.17,CONDIMENTI_CONSERVE
1,2,Ceci precotti (peso sgocciolato),Valfrutta Ceci 360 g,5.17,FRUTTA_VERDURA
2,4,Ceci precotti (peso sgocciolato),Carrefour Ceci cotti al Vapore 3 x 150 g,4.74,CONDIMENTI_CONSERVE
3,5,Ceci precotti (peso sgocciolato),Carrefour Ceci cotti al Vapore 3 x 150 g,4.74,FRUTTA_VERDURA
4,6,Ceci precotti (peso sgocciolato),Carrefour Ceci 330 g,5.07,CONDIMENTI_CONSERVE
...,...,...,...,...,...
4476,4975,Mele Golden,Mele Golden 900 g,1.99,FRUTTA_VERDURA
4477,4976,Mele Golden,Mele Golden 3 kg,1.33,FRUTTA_VERDURA
4478,4977,Mele Golden,Mele golden Melinda 3 kg,1.79,FRUTTA_VERDURA
4479,4978,Mele Golden,Mele Golden,0.98,FRUTTA_VERDURA


In [ ]:
df_grouped = data.groupby('ingrediente').agg(
    price_max=('price', 'max'),
    price_min=('price', 'min')
).reset_index()

In [ ]:
df_grouped

,ingrediente,price_max,price_min
0,"""Acciughe sottolio",49.83,49.08
1,"""Acciughe sottolio filetti",47.60,38.40
2,"""Burrata (4 da 125 g luna)",20.72,13.52
3,"""Cosciotto dagnello",3.99,3.99
4,"""Filetto di salmone da 200 g luno",NaN,NaN
...,...,...,...
858,macinato grossolanamente),NaN,NaN
859,o di pesce volante),NaN,NaN
860,pulito e rifilato,NaN,NaN
861,pulito e rifilato),NaN,NaN


In [ ]:
#df_grouped.to_csv('range_prezzi1.csv',index=False)

# FIND VALUES FOR NULL

In [ ]:
#nulli=df_merged[df_merged['corrispondenza'].isnull()]
#nulli.to_csv('nulli.csv',index=False)

In [ ]:
df_ingredienti=pd.read_csv('nulli.csv')
df_prodotti = pd.read_csv('prodotti_concatenati.csv')

Toglo il filtro di alcune parole dei brand presenti nei valori nulli

In [ ]:
lista_ingredienti = df_ingredienti['ingrediente'].tolist()

# Creare un set di parole singole
set_parole_singole = set()

for ingrediente in lista_ingredienti:
    parole = ingrediente.lower().split()  # Separare le parole singole
    set_parole_singole.update(parole)

In [ ]:
set_parole_singole

{'"filetto',
 '"francesino"',
 '"peperoncini',
 '"peperoni',
 '"petto',
 '"piadine',
 '"pomodori',
 '"scorza',
 '"succo',
 '"tonno',
 '"vacherin',
 '"zenzero',
 '(',
 '(1',
 '(2',
 '(a',
 '(abbattuto)',
 '(affumicata)',
 '(circa',
 '(con',
 '(congelato',
 '(da',
 '(diametro',
 '(facoltativo)',
 '(fresche)',
 '(gallinacci)',
 '(ghiacciata)',
 '(già',
 '(la',
 '(mini',
 '(mix',
 '(non',
 '(o',
 '(per',
 '(peso',
 '(pleurotus)',
 '(provola)',
 '(rossi',
 '(songino)',
 '(succedaneo',
 '(tipo',
 '+',
 '00',
 '1',
 '11',
 '19x21)',
 '20',
 '200',
 '28',
 '3',
 '4',
 '400',
 '8',
 'a',
 'abbattuto',
 'aceto',
 'acqua',
 'aglio',
 'agnello',
 'ai',
 'al',
 'albumi',
 'alga',
 'allantica',
 'allolio',
 'alluovo',
 'alto',
 'ambiente',
 'ambiente)',
 'anacardi',
 'aneto',
 'anice',
 'appenzeller',
 'arabo',
 'aragosta',
 'arrosto',
 'arrotolabili',
 'asiago',
 'astice',
 'avocado',
 'baby',
 'baccalà',
 'baccelli',
 'bacche',
 'baguette)',
 'bambù',
 'barbabietole',
 'basmati)',
 'bastoncini',
 

In [ ]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


# Definisci le parole inutili come un set per ricerche più veloci
PAROLE_INUTILI = set([
    'il', 'lo', 'la', 'i', 'gli', 'le', 'un', 'uno', 'una', 'del', 'dello', 'della', 'dei', 'degli', 'delle', 'e', 'di',
    'con', 'per', 'al', 'a', 'da', 'in', 'su', 'tra', 'fra', 'ma', 'anche', 'o', 'né', 'neanche',
    'nondimeno', 'tuttavia', 'anzi', 'benché', 'sebbene', 'temperatura', 'ambiente', 'fresco', 'Macinata',
    'baccello', 'Classic', 'macinato', 'grani', 'sacchetto', 'Filiera', 'Qualità', 'bio', 'fiocchi',
    'Mercato', 'sgocciolato'
])

# Estrarre parole dei brand
brand_parole = set()
for brand in df_prodotti['brand'].unique():
    brand_parole.update(re.findall(r'\b\w+\b', brand.lower()))

parole_inutili_brand = PAROLE_INUTILI | brand_parole
parole_inutili_brand -= set_parole_singole

def pulisci_testo(test: str, parole_inutili: set) -> str:
    # Rimuove tutto tra parentesi
    parole = re.sub(r'\([^)]*\)', '', test)
    # Divide la stringa in parole e converte a minuscolo
    parole = parole.lower().split()
    # Filtra le parole inutili ignorando la capitalizzazione
    parole_filtrate = [parola for parola in parole if parola not in parole_inutili]
    return ' '.join(parole_filtrate)

# Pulire ingredienti e prodotti
df_ingredienti['nome_pulito'] = df_ingredienti['ingrediente'].apply(lambda x: pulisci_testo(x, PAROLE_INUTILI))
df_prodotti['nome_pulito'] = df_prodotti['product_name'].apply(lambda x: pulisci_testo(x,parole_inutili_brand))

# Precalcolare i vettori per gli ingredienti e i prodotti usando TfidfVectorizer
vectorizer = TfidfVectorizer()
ingredienti_vettori = vectorizer.fit_transform(df_ingredienti['nome_pulito'])
prodotti_vettori = vectorizer.transform(df_prodotti['nome_pulito'])

# Lista per tenere traccia delle righe del DataFrame risultante
righe_df = []

# Itera attraverso ogni ingrediente e prodotto usando cosine similarity
similarita = cosine_similarity(ingredienti_vettori, prodotti_vettori)

for i, ingrediente in enumerate(df_ingredienti['ingrediente']):
    corrispondenze = []
    for j, prodotto in enumerate(df_prodotti['product_name']):
        indice_similarita = similarita[i, j]
        if indice_similarita > 0.4:
            corrispondenze.append({'prodotto': prodotto, 'indice_similarita': indice_similarita})

    if not corrispondenze:
        righe_df.append({'ingrediente': ingrediente, 'corrispondenza': None, 'indice_similarita': None})
    else:
        for corrispondenza in corrispondenze:
            righe_df.append({'ingrediente': ingrediente, 'corrispondenza': corrispondenza['prodotto'], 'indice_similarita': corrispondenza['indice_similarita']})

# Costruisci un DataFrame dalle righe
df_merged = pd.DataFrame(righe_df)

# Visualizza il DataFrame risultante
df_merged.head


<bound method NDFrame.head of                                              ingrediente  \
0                                        Pomodori ramati   
1                                           Garam Masala   
2                                      Cumino in polvere   
3                                      Cumino in polvere   
4                                      Cumino in polvere   
...                                                  ...   
11670  Uova di lompo (succedaneo caviale) o di salmon...   
11671  Uova di lompo (succedaneo caviale) o di salmon...   
11672  Uova di lompo (succedaneo caviale) o di salmon...   
11673  Uova di lompo (succedaneo caviale) o di salmon...   
11674  Uova di lompo (succedaneo caviale) o di salmon...   

                                          corrispondenza  indice_similarita  
0                           Pomodori ciliegini Bio 500 g           0.591219  
1                sabita Tikka Masala Simmer Sauce 200 ml           0.500000  
2              

Per diminuire il numero di errori e di osservazioni prendo solo quelli con il massimo indice di similarità

In [ ]:
def get_max_rows(group):
    max_value = group['indice_similarita'].max()
    return group[group['indice_similarita'] == max_value]

In [ ]:
max_rows = df_merged.groupby('ingrediente').apply(get_max_rows).reset_index(drop=True)

In [ ]:
max_rows

,ingrediente,corrispondenza,indice_similarita
0,"""Filetto di salmone da 200 g luno",Capitan Findus 2 Fiori di Salmone 200 g,0.698863
1,"""Filetto di salmone da 200 g luno",Capitan Findus 8 Bastoncini di Salmone 200 g,0.698863
2,"""Filetto di salmone da 200 g luno",FRoSTA Burger Dorati di Salmone Selvaggio 200 g,0.698863
3,"""Filetto di salmone da 200 g luno",Ritagli di salmone norvegese 200 g,0.698863
4,"""Filetto di salmone da 200 g luno",Gimar Smoked Salmone Affumicato 200 g,0.698863
...,...,...,...
3041,o di pesce volante),Trancio di pesce spada decongelato,0.673639
3042,o di pesce volante),Carpaccio di pesce spada 100 g,0.673639
3043,o di pesce volante),Selezione Gustosi di Mare il Consommè di Pesce...,0.673639
3044,pulito e rifilato,1 x Branzino pulito allevato Italia,0.688806


VALUES THAT ARE STILL NULL

In [ ]:
df_merged[df_merged['corrispondenza'].isnull()]

,ingrediente,corrispondenza,indice_similarita
814,Tuorli,None,NaN
1253,Biete,None,NaN
1348,Soppressata,None,NaN
1667,Chorizo,None,NaN
1734,Groviera,None,NaN
1735,Lampredotto,None,NaN
1739,Cumino,None,NaN
1990,Finocchietto selvatico,None,NaN
2294,Alga kombu,None,NaN
2714,Berberè,None,NaN


## CREATION RANGE PRICE FILE FOR NEW INGREDIENTS

I apply the same transformations made to the original dataset and then replace the null rows of the original dataset.

In [ ]:
df_matched = max_rows
df_prodotti_concatenati = pd.read_csv('prodotti_concatenati.csv')


In [ ]:
# Effettua il merge tenendo solo le osservazioni left
df_ing_prezzi = pd.merge(df_matched, df_prodotti_concatenati, left_on='corrispondenza', right_on='product_name', how='left')
df_ing_prezzi

,ingrediente,corrispondenza,indice_similarita,product_name,brand,price,source_file
0,"""Filetto di salmone da 200 g luno",Capitan Findus 2 Fiori di Salmone 200 g,0.698863,Capitan Findus 2 Fiori di Salmone 200 g,Capitan Findus,"44,95",GAELATI E SURGELATI
1,"""Filetto di salmone da 200 g luno",Capitan Findus 8 Bastoncini di Salmone 200 g,0.698863,Capitan Findus 8 Bastoncini di Salmone 200 g,Capitan Findus,"21,95",GAELATI E SURGELATI
2,"""Filetto di salmone da 200 g luno",FRoSTA Burger Dorati di Salmone Selvaggio 200 g,0.698863,FRoSTA Burger Dorati di Salmone Selvaggio 200 g,FRoSTA,"17,45",GAELATI E SURGELATI
3,"""Filetto di salmone da 200 g luno",Ritagli di salmone norvegese 200 g,0.698863,Ritagli di salmone norvegese 200 g,The Icelander,"19,95",PESCE
4,"""Filetto di salmone da 200 g luno",Gimar Smoked Salmone Affumicato 200 g,0.698863,Gimar Smoked Salmone Affumicato 200 g,Gimar,"74,50",PESCE
...,...,...,...,...,...,...,...
3331,o di pesce volante),Trancio di pesce spada decongelato,0.673639,Trancio di pesce spada decongelato,Pesce,"29,90",PESCE
3332,o di pesce volante),Carpaccio di pesce spada 100 g,0.673639,Carpaccio di pesce spada 100 g,Foodlab,"69,00",PESCE
3333,o di pesce volante),Selezione Gustosi di Mare il Consommè di Pesce...,0.673639,Selezione Gustosi di Mare il Consommè di Pesce...,Selezione Gustosi di Mare,"69,75",PESCE
3334,pulito e rifilato,1 x Branzino pulito allevato Italia,0.688806,1 x Branzino pulito allevato Italia,Filiera Qualità Carrefour,"19,90",PESCE


In [ ]:
df_ing_prezzi= df_ing_prezzi.drop(columns=['corrispondenza', 'indice_similarita', 'brand'])
df_ing_prezzi

,ingrediente,product_name,price,source_file
0,"""Filetto di salmone da 200 g luno",Capitan Findus 2 Fiori di Salmone 200 g,"44,95",GAELATI E SURGELATI
1,"""Filetto di salmone da 200 g luno",Capitan Findus 8 Bastoncini di Salmone 200 g,"21,95",GAELATI E SURGELATI
2,"""Filetto di salmone da 200 g luno",FRoSTA Burger Dorati di Salmone Selvaggio 200 g,"17,45",GAELATI E SURGELATI
3,"""Filetto di salmone da 200 g luno",Ritagli di salmone norvegese 200 g,"19,95",PESCE
4,"""Filetto di salmone da 200 g luno",Gimar Smoked Salmone Affumicato 200 g,"74,50",PESCE
...,...,...,...,...
3331,o di pesce volante),Trancio di pesce spada decongelato,"29,90",PESCE
3332,o di pesce volante),Carpaccio di pesce spada 100 g,"69,00",PESCE
3333,o di pesce volante),Selezione Gustosi di Mare il Consommè di Pesce...,"69,75",PESCE
3334,pulito e rifilato,1 x Branzino pulito allevato Italia,"19,90",PESCE


In [ ]:
df_ing_prezzi['price']=df_ing_prezzi['price'].str.replace('.', '').str.replace(',', '.').astype(float)
df_ing_prezzi

/var/folders/wl/qhbk6mrs6dx_2nxjr_p99fj00000gn/T/ipykernel_6800/3282007640.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_ing_prezzi['price']=df_ing_prezzi['price'].str.replace('.', '').str.replace(',', '.').astype(float)


,ingrediente,product_name,price,source_file
0,"""Filetto di salmone da 200 g luno",Capitan Findus 2 Fiori di Salmone 200 g,44.95,GAELATI E SURGELATI
1,"""Filetto di salmone da 200 g luno",Capitan Findus 8 Bastoncini di Salmone 200 g,21.95,GAELATI E SURGELATI
2,"""Filetto di salmone da 200 g luno",FRoSTA Burger Dorati di Salmone Selvaggio 200 g,17.45,GAELATI E SURGELATI
3,"""Filetto di salmone da 200 g luno",Ritagli di salmone norvegese 200 g,19.95,PESCE
4,"""Filetto di salmone da 200 g luno",Gimar Smoked Salmone Affumicato 200 g,74.50,PESCE
...,...,...,...,...
3331,o di pesce volante),Trancio di pesce spada decongelato,29.90,PESCE
3332,o di pesce volante),Carpaccio di pesce spada 100 g,69.00,PESCE
3333,o di pesce volante),Selezione Gustosi di Mare il Consommè di Pesce...,69.75,PESCE
3334,pulito e rifilato,1 x Branzino pulito allevato Italia,19.90,PESCE


In [ ]:
df_grouped = df_ing_prezzi.groupby('ingrediente').agg(
    price_max=('price', 'max'),
    price_min=('price', 'min')
).reset_index()
df_grouped

,ingrediente,price_max,price_min
0,"""Filetto di salmone da 200 g luno",74.50,17.45
1,"""Peperoncini Jalapeno sottaceto",13.93,13.93
2,"""Peperoni sottolio arrosto",49.75,6.90
3,"""Petto danatra",29.83,10.68
4,"""Piadine Sfogliatissime allOlio EVO",12.50,3.65
...,...,...,...
310,Zucchine piccole,5.62,5.62
311,Zucchine tonde,10.68,1.59
312,o di pesce volante),69.75,6.64
313,pulito e rifilato,19.90,19.90


In [ ]:
#df_grouped.to_csv('not_nulli.csv',index=False)

# MERGE NEW INGREDIENTS TO COVER NULL VALUES

sostituisco le righe nulle del dataset originale con le righe del nuovo dataset

In [ ]:
data=pd.read_csv('range_prezzi1.csv')
new_data=pd.read_csv('not_nulli.csv')

In [ ]:
data

,ingrediente,price_max,price_min
0,"""Acciughe sottolio",49.83,49.08
1,"""Acciughe sottolio filetti",47.60,38.40
2,"""Burrata (4 da 125 g luna)",20.72,13.52
3,"""Cosciotto dagnello",3.99,3.99
4,"""Filetto di salmone da 200 g luno",NaN,NaN
...,...,...,...
858,macinato grossolanamente),NaN,NaN
859,o di pesce volante),NaN,NaN
860,pulito e rifilato,NaN,NaN
861,pulito e rifilato),NaN,NaN


In [ ]:
# Imposta l'indice su 'ingrediente' per facilitare la sostituzione
data.set_index('ingrediente', inplace=True)
new_data.set_index('ingrediente', inplace=True)

In [ ]:
# Funzione per sostituire le righe con valori nulli
def replace_nulls(row, df2):
    if row.isnull().any():
        ingrediente = row.name
        if ingrediente in df2.index:
            row = df2.loc[ingrediente]
    return row

In [ ]:
# Applicare la funzione per sostituire le righe con valori nulli
filled_df = data.apply(lambda row: replace_nulls(row, new_data), axis=1)
# Ripristina l'indice come colonna
filled_df.reset_index(inplace=True)
filled_df

,ingrediente,price_max,price_min
0,"""Acciughe sottolio",49.83,49.08
1,"""Acciughe sottolio filetti",47.60,38.40
2,"""Burrata (4 da 125 g luna)",20.72,13.52
3,"""Cosciotto dagnello",3.99,3.99
4,"""Filetto di salmone da 200 g luno",74.50,17.45
...,...,...,...
858,macinato grossolanamente),NaN,NaN
859,o di pesce volante),69.75,6.64
860,pulito e rifilato,19.90,19.90
861,pulito e rifilato),19.90,19.90


In [ ]:
filled_df[filled_df['price_max'].isnull()]

,ingrediente,price_max,price_min
49,Albumi,NaN,NaN
51,Alga kombu,NaN,NaN
61,Appenzeller,NaN,NaN
65,Aragosta 4,NaN,NaN
72,Astice (congelato già cotto),NaN,NaN
91,Berberè,NaN,NaN
93,Biete,NaN,NaN
94,Bietole,NaN,NaN
95,Bietole (fresche),NaN,NaN
124,Cajun,NaN,NaN


In [ ]:
#filled_df.to_csv('filled_data.csv', index=False)

# JSON CREATION

### TRASFORMATION OF KG PRICES TO G

In [ ]:
import pandas as pd

In [ ]:
data=pd.read_csv('filled_data.csv')

In [ ]:
data['price_max']= data['price_max']/1000

In [ ]:
data['price_min']= data['price_min']/1000

In [ ]:
#data.to_csv('prezzi_grammi.csv')

### ADD THE PRICES TO THE JSON FILE

In [ ]:
import json
import pandas as pd
import numpy as np

# Carica il file JSON
with open('/Volumes/T7 Shield/SCUOLA/DATA MANEGEMENT PROGETTO/BELLA PROGETTO/progetto data man/FINALE/Ingredients_cleaned.json', 'r') as f:
    recipes = json.load(f)

# Carica il file CSV
prices = pd.read_csv('prezzi_grammi.csv')

# Funzione per ottenere il prezzo totale
def get_price(ingredient, quantity, prices_df):
    price_row = prices_df[prices_df['ingrediente'] == ingredient]
    if not price_row.empty:
        price_max = price_row['price_max'].values[0]
        price_min = price_row['price_min'].values[0]
        total_price_max = (quantity) * price_max  # quantità in grammi
        total_price_min = (quantity) * price_min  # quantità in grammi
        return total_price_max, total_price_min
    return np.nan, np.nan

# Aggiorna il file JSON con i prezzi
for recipe in recipes:
    for ingredient, quantity_str in recipe['ingredienti'].items():
        quantity = float(quantity_str.split()[0])  # Estrae la quantità come numero
        price_max, price_min = get_price(ingredient, quantity, prices)
        recipe['ingredienti'][ingredient] = {
            'quantity': quantity_str,
            'price_max': price_max,
            'price_min': price_min
        }

# Salva il nuovo file JSON
with open('ingredients_with_prices.json', 'w') as f:
    json.dump(recipes, f, ensure_ascii=False, indent=4)


### ADD TOT_PRICE FOR RECIPE

In [ ]:
import json
import math

# Carica il file JSON
with open('/Volumes/T7 Shield/SCUOLA/DATA MANEGEMENT PROGETTO/BELLA PROGETTO/progetto data man/FINALE/ingredients_with_prices.json', 'r') as file:
    data = json.load(file)

# Funzione per calcolare il totale di price_max e price_min per una ricetta
def calculate_total_prices(recipe):
    total_price_max = 0
    total_price_min = 0
    for ingredient_name, ingredient_details in recipe['ingredienti'].items():
        try:
            price_max = ingredient_details['price_max']
            price_min = ingredient_details['price_min']
            if not math.isnan(price_max):  # Verifica se il valore non è NaN
                total_price_max += price_max
            if not math.isnan(price_min):  # Verifica se il valore non è NaN
                total_price_min += price_min
        except KeyError as e:
            print(f"Errore: {e} non trovato per l'ingrediente {ingredient_name}")
        except TypeError as e:
            print(f"Errore: {e} per l'ingrediente {ingredient_name} con details {ingredient_details}")
    return total_price_max, total_price_min

# Itera su ogni ricetta e calcola i totali di price_max e price_min
for recipe in data:
    total_price_max, total_price_min = calculate_total_prices(recipe)
    recipe["tot_price_max"] = total_price_max
    recipe["tot_price_min"] = total_price_min

# Salva il nuovo file JSON con i totali aggiornati
with open('ingredients_with_tot_prices.json', 'w') as file:
    json.dump(data, file, ensure_ascii=False, indent=4)

# Stampa i risultati (opzionale)
for recipe in data:
    print(f"Il totale di price_max per '{recipe['titolo']}' è: {recipe['tot_price_max']}")
    print(f"Il totale di price_min per '{recipe['titolo']}' è: {recipe['tot_price_min']}")


# IMPORT THE FILE TO MONGODB

In [ ]:
import json
from pymongo import MongoClient

# Path del file JSON
file_path = '/Volumes/T7 Shield/SCUOLA/DATA MANEGEMENT PROGETTO/BELLA PROGETTO/progetto data man/FINALE/ingredients_with_tot_prices.json'

# Connessione a MongoDB (aggiorna la stringa di connessione se necessario)
client = MongoClient('mongodb://localhost:27017')

# Nome del database e della collezione
db = client['Progetto_data_man']
collection = db['Ricette_tot_prezzo']

# Lettura del file JSON
with open(file_path, 'r') as file:
    data = json.load(file)

# Inserimento dei dati nella collezione
if isinstance(data, list):
    collection.insert_many(data)
else:
    collection.insert_one(data)

print("Dati importati con successo!")ch

# IMPORT CALORIES ON MONGODB

In [ ]:
import json
from pymongo import MongoClient
import pandas as pd

In [ ]:
data=pd.read_csv('porzioni_calorie_sistemato.csv')

In [ ]:
client = MongoClient('mongodb://localhost:27017/')
db = client['Progetto_data_man']  # Sostituisci con il nome del tuo database
collection = db['calories_portions']  # Sostituisci con il nome della tua collezione

# Trasformazione dei dati in formato JSON e inserimento in MongoDB
data_dict = data.to_dict(orient='records')  # Converte il DataFrame in una lista di dizionari

# Inserisce i dati nella collezione di MongoDB
collection.insert_many(data_dict)


print("Dati importati correttamente in MongoDB.")

Dati importati correttamente in MongoDB.
